## Votação Seção


```mysql
create table votacao_secao_2006(
    DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    CODIGO_MUNICIPIO STRING,
    NOME_MUNICIPIO STRING,
    NUM_ZONA STRING,
    NUM_SECAO STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    NUM_VOTAVEL STRING,
    QTDE_VOTOS STRING) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"',
'serialization.encoding'='ISO-8859-1'
)
STORED AS TEXTFILE;
Alter table votacao_secao_2006 set location 's3://datascience2016/data/gzip/votacao_secao/2006/';
```



## Candidato

```mysql
create table candidato_2006 
    (DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    DESCRICAO_UE STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    NOME_CANDIDATO STRING,
    SEQUENCIAL_CANDIDATO STRING,
    NUMERO_CANDIDATO STRING,
    CPF_CANDIDATO STRING,
    NOME_URNA_CANDIDATO STRING,
    COD_SITUACAO_CANDIDATURA STRING,
    DES_SITUACAO_CANDIDATURA STRING,
    NUMERO_PARTIDO STRING,
    SIGLA_PARTIDO STRING,
    NOME_PARTIDO STRING,
    CODIGO_LEGENDA STRING,
    SIGLA_LEGENDA STRING,
    COMPOSICAO_LEGENDA STRING,
    NOME_LEGENDA STRING,
    CODIGO_OCUPACAO STRING,
    DESCRICAO_OCUPACAO STRING,
    DATA_NASCIMENTO STRING,
    NUM_TITULO_ELEITORAL_CANDIDATO STRING,
    IDADE_DATA_ELEICAO STRING,
    CODIGO_SEXO STRING,
    DESCRICAO_SEXO STRING,
    COD_GRAU_INSTRUCAO STRING,
    DESCRICAO_GRAU_INSTRUCAO STRING,
    CODIGO_ESTADO_CIVIL STRING,
    DESCRICAO_ESTADO_CIVIL STRING,
    CODIGO_COR_RACA STRING,
    DESCRICAO_COR_RACA STRING,
    CODIGO_NACIONALIDADE STRING,
    DESCRICAO_NACIONALIDADE STRING,
    SIGLA_UF_NASCIMENTO STRING,
    CODIGO_MUNICIPIO_NASCIMENTO STRING,
    NOME_MUNICIPIO_NASCIMENTO STRING,
    DESPESA_MAX_CAMPANHA STRING,
    COD_SIT_TOT_TURNO STRING,
    DESC_SIT_TOT_TURNO STRING,
    NM_EMAIL STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"',
'serialization.encoding'='ISO-8859-1'
)
STORED AS TEXTFILE;
Alter table candidato_2006 set location 's3://datascience2016/data/gzip/candidatos/2006/';
```


### Legenda


```mysql
create table legenda_2006(
    DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    NOME_UE STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    TIPO_LEGENDA STRING,
    NUM_PARTIDO STRING,
    SIGLA_PARTIDO STRING,
    NOME_PARTIDO STRING,
    SIGLA_COLIGACAO STRING,
    NOME_COLIGACAO STRING,
    COMPOSICAO_COLIGACAO STRING,
    SEQUENCIA_COLIGACAO STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"'
)
STORED AS TEXTFILE;
Alter table legenda_2006 set location 's3://datascience2016/data/gzip/legendas/2006/';
```





## Candidato - legenda
```mysql
    create view candidato_2006_legenda as
    select 
    c.*,
    l.TIPO_LEGENDA,
    l.SIGLA_COLIGACAO,
    l.NOME_COLIGACAO,
    l.COMPOSICAO_COLIGACAO,
    l.SEQUENCIA_COLIGACAO
    from candidato_2006 as c 
    left join legenda_2006 as l 
    on c.SIGLA_UE=l.SIGLA_UE 
    AND c.NUMERO_PARTIDO=l.NUM_PARTIDO 
    AND c.CODIGO_CARGO=l.CODIGO_CARGO;
```



# Agregações Regionais


### macro
```mysql
create view votacao_secao_2006_macro as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UF,
CODIGO_MACRO,
NOME_MACRO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO
from (select * from votacao_secao_2006 where CODIGO_CARGO<>"1") v left join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by 
ANO_ELEICAO,
NUM_TURNO,
SIGLA_UF,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO;

```

### UF
```mysql
create view votacao_secao_2006_uf as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF 
from (select * from votacao_secao_2006 where CODIGO_CARGO<>"1") v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF;

```
### meso

```mysql
create view votacao_secao_2006_meso as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO 
from (select * from votacao_secao_2006 where CODIGO_CARGO<>"1") v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
SIGLA_UE,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO;
```
### micro

```mysql
create view votacao_secao_2006_micro as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO,m.CODIGO_MICRO,m.NOME_MICRO 
from (select * from votacao_secao_2006 where CODIGO_CARGO<>"1") v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
UF,
NOME_UF,
SIGLA_UE,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO;

```

### Municipio


```mysql
create view votacao_secao_2006_mun as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO,m.CODIGO_MICRO,m.NOME_MICRO
from (select * from votacao_secao_2006 where CODIGO_CARGO<>"1") v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
UF,
NOME_UF,
SIGLA_UE,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO;

```

### Local de votacao


```mysql
create view votacao_secao_2006_local as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
SIGLA_UF,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO,
max(cast(NUM_ZONA as INT)) as NUM_ZONA,
NUMERO_LOCAL_VOTACAO,
NOME_LOCAL_VOTACAO,
ENDERECO_LOCAL_VOTACAO,
CEP_LOCAL_VOTACAO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.NUMERO_LOCAL_VOTACAO,
m.NOME_LOCAL_VOTACAO,
m.ENDERECO_LOCAL_VOTACAO,
m.CEP_LOCAL_VOTACAO
from (select * from votacao_secao_2006 where CODIGO_CARGO<>"1") as v left join aux_local_votacao as m on 
m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO AND
m.NUM_ZONA=v.NUM_ZONA AND
m.NUM_SECAO=v.NUM_SECAO) as v_s_m
group by
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
sigla_uf,
sigla_UE,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO,
NUMERO_LOCAL_VOTACAO,
NOME_LOCAL_VOTACAO,
ENDERECO_LOCAL_VOTACAO,
CEP_LOCAL_VOTACAO;
```




## join com os candidatos

```mysql

create view votacao_secao_2006_macro_candidato as
select
    c.ANO_ELEICAO,
    v.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    v.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    v.NUM_VOTAVEL ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.QTDE_VOTOS
from votacao_secao_2006_macro as v
left join candidato_2006_legenda as c
on v.SIGLA_UF=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;

create view votacao_secao_2006_uf_candidato as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.QTDE_VOTOS
from votacao_secao_2006_uf as v
left join candidato_2006_legenda as c
on v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.UF=c.SIGLA_UE
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;



create view votacao_secao_2006_meso_candidato as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.UF,
    v.NOME_UF,
    v.QTDE_VOTOS
from votacao_secao_2006_meso as v
join candidato_2006_legenda as c
on v.UF=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;


create view votacao_secao_2006_micro_candidato as
select
    v.ANO_ELEICAO,
    v.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    v.SIGLA_UE,
    c.DESCRICAO_UE ,
    v.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.CODIGO_MICRO,
    v.NOME_MICRO,
    v.QTDE_VOTOS
from votacao_secao_2006_micro as v
join candidato_2006_legenda as c
on v.UF=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;


create view votacao_secao_2006_mun_candidato as
select
    v.ANO_ELEICAO,
    v.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    v.SIGLA_UE,
    c.DESCRICAO_UE ,
    v.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.CODIGO_MICRO,
    v.NOME_MICRO,
    v.CODIGO_MUNICIPIO,
    v.NOME_MUNICIPIO,
    v.QTDE_VOTOS
from votacao_secao_2006_mun as v
left join candidato_2006_legenda as c
on v.UF=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;


create view votacao_secao_2006_local_candidato as
select
    v.ANO_ELEICAO,
    v.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    v.SIGLA_UE,
    c.DESCRICAO_UE ,
    v.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MUNICIPIO,
    v.NOME_MUNICIPIO,
    v.NUM_ZONA,
    v.NUMERO_LOCAL_VOTACAO,
    v.NOME_LOCAL_VOTACAO,
    v.ENDERECO_LOCAL_VOTACAO,
    v.CEP_LOCAL_VOTACAO,
    v.QTDE_VOTOS
from votacao_secao_2006_local as v
left join candidato_2006_legenda as c
on v.SIGLA_UE=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;




```

```mysql

create view votacao_secao_2006_macro_presidente as
select v.ANO_ELEICAO,
    v.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    v.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    v.NUM_VOTAVEL as NUMERO_CANDIDATO,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.QTDE_VOTOS
from (select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO
from (select * from votacao_secao_2006 where CODIGO_CARGO="1") v left join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO) as v
left join candidato_2006_legenda as c
on v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;


create view votacao_secao_2006_uf_presidente as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.QTDE_VOTOS
from (select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF 
from (select * from votacao_secao_2006 where CODIGO_CARGO="1") v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF) as v
left join candidato_2006_legenda as c
on v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;



create view votacao_secao_2006_meso_presidente as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.UF,
    v.NOME_UF,
    v.QTDE_VOTOS
from (select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO 
from (select * from votacao_secao_2006 where CODIGO_CARGO="1") v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO) as v
join candidato_2006_legenda as c
on v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;


create view votacao_secao_2006_micro_presidente as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.CODIGO_MICRO,
    v.NOME_MICRO,
    v.QTDE_VOTOS
from (select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO,m.CODIGO_MICRO,m.NOME_MICRO 
from (select * from votacao_secao_2006 where CODIGO_CARGO="1") v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
UF,
NOME_UF,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO) as v
join candidato_2006_legenda as c
on v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;

create view votacao_secao_2006_mun_presidente as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.CODIGO_MICRO,
    v.NOME_MICRO,
    v.CODIGO_MUNICIPIO,
    v.NOME_MUNICIPIO,
    v.QTDE_VOTOS
from (select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO,m.CODIGO_MICRO,m.NOME_MICRO
from (select * from votacao_secao_2006 where CODIGO_CARGO="1") v left join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
UF,
NOME_UF,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO) as v
left join candidato_2006_legenda as c
on v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;



create view votacao_secao_2006_local_presidente as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    "#NE#" as CODIGO_COR_RACA,
    "#NE#" as DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    "#NE#" as NM_EMAIL,
    c.TIPO_LEGENDA,
    c.SIGLA_COLIGACAO,
    c.NOME_COLIGACAO,
    c.COMPOSICAO_COLIGACAO,
    c.SEQUENCIA_COLIGACAO,
    v.CODIGO_MUNICIPIO,
    v.NOME_MUNICIPIO,
    v.NUM_ZONA,
    v.NUMERO_LOCAL_VOTACAO,
    v.NOME_LOCAL_VOTACAO,
    v.ENDERECO_LOCAL_VOTACAO,
    v.CEP_LOCAL_VOTACAO,
    v.QTDE_VOTOS
from (select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UF,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO,
max(cast(NUM_ZONA as INT)) as NUM_ZONA,
NUMERO_LOCAL_VOTACAO,
NOME_LOCAL_VOTACAO,
ENDERECO_LOCAL_VOTACAO,
CEP_LOCAL_VOTACAO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.NUMERO_LOCAL_VOTACAO,
m.NOME_LOCAL_VOTACAO,
m.ENDERECO_LOCAL_VOTACAO,
m.CEP_LOCAL_VOTACAO
from (select * from votacao_secao_2006 where CODIGO_CARGO="1") as v left join aux_local_votacao as m on 
m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO AND
m.NUM_ZONA=v.NUM_ZONA AND
m.NUM_SECAO=v.NUM_SECAO) as v_s_m
group by
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
sigla_uf,
CODIGO_MUNICIPIO,
NOME_MUNICIPIO,
NUMERO_LOCAL_VOTACAO,
NOME_LOCAL_VOTACAO,
ENDERECO_LOCAL_VOTACAO,
CEP_LOCAL_VOTACAO) as v
left join candidato_2006_legenda as c
on v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;
```
